In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime

# Define assets
TICKERS = ['AAPL', '^GSPC']
START_DATE = datetime(2020, 1, 1)
END_DATE = datetime.now()

# 1. Fetch data
price_data = yf.download(TICKERS, start=START_DATE, end=END_DATE, auto_adjust=False)['Adj Close']

# 2. Calculate returns
daily_returns = price_data.pct_change().dropna()

# 3. Rename columns
data = daily_returns.rename(columns={
    'AAPL': 'Asset_A_Returns',
    '^GSPC': 'Index_B_Returns'
})

# Display output
print("\n--- Daily Returns DataFrame ('data') ---")
print(data.head())
print(data)

[*********************100%***********************]  2 of 2 completed


--- Daily Returns DataFrame ('data') ---
Ticker      Asset_A_Returns  Index_B_Returns
Date                                        
2020-01-03        -0.009722        -0.007060
2020-01-06         0.007968         0.003533
2020-01-07        -0.004703        -0.002803
2020-01-08         0.016086         0.004902
2020-01-09         0.021241         0.006655
Ticker      Asset_A_Returns  Index_B_Returns
Date                                        
2020-01-03        -0.009722        -0.007060
2020-01-06         0.007968         0.003533
2020-01-07        -0.004703        -0.002803
2020-01-08         0.016086         0.004902
2020-01-09         0.021241         0.006655
...                     ...              ...
2025-11-10         0.004549         0.015401
2025-11-11         0.021601         0.002075
2025-11-12        -0.006467         0.000630
2025-11-13        -0.001901        -0.016557
2025-11-14         0.003688        -0.000098

[1476 rows x 2 columns]


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

# --- 1. Download data (disable auto_adjust if you want raw Close prices) ---
tickers = ['AAPL', 'SPY']
data = yf.download(tickers, start='2023-01-01', end='2024-01-01', auto_adjust=False)['Close']

# --- 2. Calculate daily returns ---
returns = data.pct_change().dropna()
returns.rename(columns={'AAPL': 'Asset_A_Returns', 'SPY': 'Index_B_Returns'}, inplace=True)

# --- 3. Compute covariance and variance ---
covariance = returns['Asset_A_Returns'].cov(returns['Index_B_Returns'])
variance = returns['Index_B_Returns'].var()

# --- 4. Compute beta ---
beta = covariance / variance

# --- 5. Print nicely ---
print(f"Covariance (Stock, Market): {covariance}")
print(f"Variance (Market): {variance}")
print(f"Calculated Beta: {beta}")


[*********************100%***********************]  2 of 2 completed

Covariance (Stock, Market): 7.537579463698884e-05
Variance (Market): 6.91981180717508e-05
Calculated Beta: 1.0892752106181913


In [ ]:
# --- CAPM Calculation (Required Return) ---
Risk_Free_Rate = 0.04103  # 4.103%

# 1. Calculate Average Index Return (Annualized)
Average_Daily_Index_Return = returns['Index_B_Returns'].mean()
Average_Index_Return = Average_Daily_Index_Return * 252  # 252 trading days

print(f"Average S&P 500 Performance (Annualized): {round(Average_Index_Return * 100, 2)}%")

# 2. Compute Required Return using CAPM
Required_Return = Risk_Free_Rate + beta * (Average_Index_Return - Risk_Free_Rate)

print(f"Required Return (CAPM): {round(Required_Return * 100, 2)}%")


Average S&P 500 Performance (Annualized): 23.31%
Required Return (CAPM): 25.02%


In [ ]:
# --- Exposure at Risk (EaR) Determination ---

# Define investment exposure and threshold
CAPITAL_EXPOSURE = 1000
CONFIDENCE_THRESHOLD = 0.99

# Compute the alpha value
ALPHA_VALUE = 1 - CONFIDENCE_THRESHOLD

# 1. Identify the worst-case return quantile (e.g., 1st percentile)
# FIX APPLIED: Changed 'data' to 'returns'
worst_case_percentile = returns['Asset_A_Returns'].quantile(ALPHA_VALUE)

# 2. Estimate the EaR in monetary units
exposure_at_risk_value = CAPITAL_EXPOSURE * worst_case_percentile

# --- Present Findings ---

print("\n--- Exposure at Risk (EaR) Determination ---")
print(f"Investment Exposure: ${CAPITAL_EXPOSURE:,.2f}")
print(f"Confidence Threshold: {CONFIDENCE_THRESHOLD:.0%}")
print(f"Worst-Case Percentile Return: {worst_case_percentile:.2%}")

# EaR is quoted as a positive figure
print("-" * 30)
print(f"Empirical EaR (1-session, {CONFIDENCE_THRESHOLD:.0%}): ${-exposure_at_risk_value:,.2f}")


--- Exposure at Risk (EaR) Determination ---
Investment Exposure: $1,000.00
Confidence Threshold: 99%
Worst-Case Percentile Return: -2.80%
------------------------------
Empirical EaR (1-session, 99%): $28.02
